In [2]:
import us
import re
import pandas as pd
import numpy as np
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser=SentimentIntensityAnalyzer()

In [ ]:
rawdf = pd.read_csv("/content/sample_data/tweets.csv")

In [ ]:
rawdf = rawdf.drop_duplicates(subset=['text', 'user_name'])

In [5]:
rawdf = rawdf.dropna(subset=['hashtags'])

NameError: ignored

In [ ]:
rawdf['date'] = pd.to_datetime(rawdf['date'], utc=True)
rawdf['tweet_date'] = rawdf['date'].dt.strftime('%d/%m/%Y')

In [ ]:
rawdf.drop('date', axis=1, inplace=True)

In [ ]:
rawdf["user_location"].value_counts()

In [ ]:
rawdf.isnull().sum()

In [ ]:
scores = []
for text in rawdf['cleaned_text'].tolist():
    score = analyser.polarity_scores(text)
    score = score['compound']
    scores.append(score)

sentiment = []
for score in scores:
    if score >= 0.05:
        sentiment.append('Positive')
    elif score <= -0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

rawdf['sentiment'] = pd.Series(sentiment)

In [4]:
def clean_tweet_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s#]', '', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

rawdf['cleaned_text'] = rawdf['text'].apply(clean_tweet_text)

NameError: ignored

In [ ]:
rawdf['tweet_length'] = rawdf['cleaned_text'].apply(len)

In [ ]:
# Calculate user account age
rawdf['user_account_age'] = (pd.to_datetime('now', utc=True) - pd.to_datetime(rawdf['user_created'])).dt.days

In [ ]:
rawdf['follower_ratio'] = rawdf['user_followers'] / rawdf['user_friends']

In [ ]:
rawdf.drop('user_location', axis=1, inplace=True)

In [ ]:
rawdf.to_csv('LLMtweets_final.csv', index=False)

In [ ]:
testdf = rawdf.head(10)

In [ ]:
def extract_first_word(lst):
    try:
        # Convert the list to a string representation and remove square brackets and quotes
        result = str(lst).strip("[]").replace("'", "").split(',')[0].strip()
        return result.upper()
    except (IndexError, AttributeError):
        return None

# Apply the modified function to create a new column 'hashtag_new'
rawdf['hashtag_new'] = rawdf['hashtags'].apply(extract_first_word)

In [ ]:
for index, user_location in rawdf['user_location'].iteritems():
    # Check if 'user_location' is not NaN
    if pd.notna(user_location):
        # Strip the value of 'user_location' on ','
        parts = user_location.split(',')

        # Check if 'user_location' contains a U.S. city or state
        us_city = us.states.lookup(parts[0].strip())
        us_state = us.states.lookup(parts[0].strip())

        if us_city or us_state:
            # Update 'standardized_country' to 'United States'
            rawdf.at[index, 'standardized_country'] = 'United States'

<ipython-input-145-8260af6a287f>:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, user_location in rawdf['user_location'].iteritems():


In [ ]:
rawdf.info()

NameError: ignored